# LangChain

In [1]:
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import os
from IPython.display import Image, display
import streamlit as st
from langchain.vectorstores import Chroma 
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader 
from langchain.document_loaders import PyPDFLoader
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.cache import InMemoryCache
import langchain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate,SystemMessagePromptTemplate,AIMessagePromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser,DatetimeOutputParser
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import SimpleSequentialChain,SequentialChain
from langchain.memory import ChatMessageHistory
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import pickle
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import numpy as np

In [2]:
def setup_environment():
    import sys
    sys.path.append('C:\\gitworkspace\\aimldemo\\jupyterworkapce')
    import stratup_env_setup
    stratup_env_setup.set_env()

In [3]:
setup_environment()

In [4]:
llm = OpenAI(temperature=0)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_13360\3183534344.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


# Basics

In [5]:
#llm('One sentence about Piscataway, NJ')
llm('One sentence about Paris')

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\2437324265.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm('One sentence about Paris')


'\n\nParis is the capital of France and known for its iconic landmarks, rich history, and romantic atmosphere.'

In [6]:
result = llm.generate(['Once sentence about Piscataway, NJ','One sentence about Edison, NJ'],max_tokens=100)

In [7]:
result

LLMResult(generations=[[Generation(text='\n\nPiscataway, NJ is a diverse and vibrant community located in central New Jersey.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nEdison, NJ is a diverse and bustling township known for its strong economy, top-rated schools, and vibrant community.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 59, 'prompt_tokens': 15, 'completion_tokens': 44}, 'model_name': 'gpt-3.5-turbo-instruct'}, run=[RunInfo(run_id=UUID('b0ef16ac-0d3b-41eb-bc6e-406b00a1a0c9')), RunInfo(run_id=UUID('0cff0bae-8aab-4027-8b68-2c72bef1cf17'))], type='LLMResult')

In [8]:
type(result)

langchain_core.outputs.llm_result.LLMResult

In [9]:
result.schema()

{'$defs': {'BaseMessage': {'additionalProperties': True,
   'description': 'Base abstract message class.\n\nMessages are the inputs and outputs of ChatModels.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type', 'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Id'}},
   'required': ['content', 'type'],
   'title': 'BaseMessage',
   'type': 'object'},
  'BaseMessageChunk': {'additionalProperties': True,
   'description': 'Message chunk, which can be concatenated with other Message chunks.',
   'properties': {'co

In [10]:
result.generations[0][0].text

'\n\nPiscataway, NJ is a diverse and vibrant community located in central New Jersey.'

In [11]:
result.generations[1][0].text

'\n\nEdison, NJ is a diverse and bustling township known for its strong economy, top-rated schools, and vibrant community.'

In [12]:
chat = ChatOpenAI()

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\3310371265.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI()


In [13]:
result = chat([HumanMessage(content='One sentence about Piscataway, NJ')])

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\1919734953.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chat([HumanMessage(content='One sentence about Piscataway, NJ')])


In [14]:
type(result)

langchain_core.messages.ai.AIMessage

In [15]:
result.content

'Piscataway, NJ is a diverse and vibrant community located in Middlesex County.'

In [16]:
result = chat([SystemMessage(content='You are a comedian'),
    HumanMessage(content='five sentencec about Piscataway, NJ')])

In [17]:
result.content

"Piscataway, NJ is like the middle child of New Jersey - often overlooked but full of hidden gems. It's a diverse community with a mix of suburban neighborhoods and bustling commercial areas. The traffic on Route 18 can be a nightmare, but at least it gives you time to appreciate the beautiful Raritan River. Residents take pride in their local sports teams, especially the Piscataway High School Chiefs. And let's not forget about the delicious food scene, with everything from classic diners to trendy restaurants."

# Embeddings

In [18]:
text = "Today is December 5, 2024"

In [19]:
embeddings = OpenAIEmbeddings()

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\2497576997.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [20]:
embedded_text = embeddings.embed_query(text)

In [21]:
type(embedded_text)

list

In [22]:
embedded_text

[-0.013985867914812407,
 -0.009167040896666867,
 -0.01763147340138314,
 -0.009717195505143078,
 -0.029509518934311547,
 -0.004172561314872382,
 -0.023795860403298924,
 -0.008603628911054026,
 -0.03409635349409849,
 0.011069383711820339,
 0.02717633045433078,
 0.004364783970127574,
 -0.0023597009059475415,
 -0.0049779085588656435,
 -0.01028060749075592,
 0.017578445755481807,
 0.020216539939766152,
 -0.014410083494087494,
 0.02681839896677106,
 -0.026301386404152534,
 0.008630141802682097,
 -0.0011027956154498234,
 -0.016769783934373738,
 0.011977470739178864,
 0.021741064735993663,
 0.0013712448131962354,
 -0.0004001880384064996,
 -0.016663730505216263,
 0.04228902327169146,
 -0.012892185989444409,
 0.014980123702607354,
 -0.018692013280893455,
 0.009670797013471358,
 -0.04112242996302368,
 -0.011884673755835426,
 -0.03218738059339281,
 -0.016690245259489526,
 -0.011937700470414163,
 0.017697757493098508,
 0.022098996223553383,
 -0.013064524441639845,
 0.020057458864707348,
 0.00402010

In [23]:
def cosine_similarity(text1, text2):
    response1 = embeddings.embed_query(text1)
    response2 = embeddings.embed_query(text2)
    similarity_score = np.dot(response1,response2)
    print(f"Cosine Similarity between inputs: {similarity_score*100}")


In [24]:
cosine_similarity(text,'What day is today?')

Cosine Similarity between inputs: 81.36025253341899


In [25]:
cosine_similarity('How are you?','How is your day?')

Cosine Similarity between inputs: 91.50557791215063


In [26]:
cosine_similarity('aaaa','aaaa1')

Cosine Similarity between inputs: 91.0319310609147


In [27]:
cosine_similarity('aaaa','hello how are?. How is your day?')

Cosine Similarity between inputs: 78.64562198723497


In [28]:
text_chunk_1 = "In 2020, home interest rates were at a historic low of 3%."
text_chunk_2 = "Interest rates dropped to 3% in 2020, encouraging homebuyers."

In [29]:
cosine_similarity(text_chunk_1,text_chunk_2)

Cosine Similarity between inputs: 93.55030765664607


In [30]:
cosine_similarity('In 2020, home interest rates were at a historic low of 3%.', 'What was the interest rate in 2020?')

Cosine Similarity between inputs: 88.45950861648248


In [31]:
cosine_similarity('In 2020, home interest rates were at a historic low of 3%.', 'What was the interest rate in 2024?')

Cosine Similarity between inputs: 83.61030890422309


In [32]:
cosine_similarity('In 2020, home interest rates were at a historic low of 3%.', 'What is the capital of France?')

Cosine Similarity between inputs: 69.07058551329177


* Embeddings with defualt Chroma Embedding

In [33]:
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import ONNXMiniLM_L6_V2

In [34]:
embeddings_chroma = ONNXMiniLM_L6_V2()

In [35]:
type(embeddings_chroma)

chromadb.utils.embedding_functions.ONNXMiniLM_L6_V2

In [36]:
def cosine_similarity_chroma(text1, text2):
    # Compute embeddings
    response1 = embeddings_chroma(text1)  # Shape: (N, 384)
    response2 = embeddings_chroma(text2)  # Shape: (M, 384)
    
    # Average the embeddings to get a single vector for each input
    embedding1 = np.mean(response1, axis=0)  # Shape: (384,)
    embedding2 = np.mean(response2, axis=0)  # Shape: (384,)
    
    # Compute cosine similarity
    similarity_score = np.dot(embedding1, embedding2) / (
        np.linalg.norm(embedding1) * np.linalg.norm(embedding2)
    )
    
    # Print similarity as a percentage
    print(f"Cosine Similarity between inputs with Chroma ONNXMiniLM_L6_V2: {similarity_score * 100:.2f}%")
    return similarity_score

In [37]:
cosine_similarity_chroma(text,'What day is today?')

Cosine Similarity between inputs with Chroma ONNXMiniLM_L6_V2: 95.38%


0.9537710304321807

* Use BERT model

In [38]:
from langchain.embeddings import HuggingFaceEmbeddings

In [39]:
embedding_bert = HuggingFaceEmbeddings(model_name="bert-base-uncased")

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\831216753.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_bert = HuggingFaceEmbeddings(model_name="bert-base-uncased")
C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU 

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


In [40]:
def cosine_similarity_BERT(text1, text2):
    # Compute embeddings
    embedding1 = embedding_bert.embed_query(text1)  # Shape: (768,)
    embedding2 = embedding_bert.embed_query(text2)  # Shape: (768,)
    
    # Compute cosine similarity
    similarity_score = np.dot(embedding1, embedding2) / (
        np.linalg.norm(embedding1) * np.linalg.norm(embedding2)
    )
    
    # Print similarity as a percentage
    print(f"Cosine Similarity between inputs with BERT Embeddings: {similarity_score * 100:.2f}%")
    return similarity_score

In [41]:
cosine_similarity_BERT(text,'What day is today?')

Cosine Similarity between inputs with BERT Embeddings: 63.48%


0.6348337236583323

# Chat Model

In [42]:
chat = ChatOpenAI()

In [43]:
result = chat([HumanMessage(content="Can you tell me a fact about Piscataway, NJ?")])

In [44]:
result

AIMessage(content="One interesting fact about Piscataway, NJ is that it is home to Rutgers University's main campus, making it one of the oldest towns in the United States with a university presence.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 21, 'total_tokens': 60, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1d4d0cb5-cc6b-477e-b28a-2f74e0636189-0')

In [45]:
result.content

"One interesting fact about Piscataway, NJ is that it is home to Rutgers University's main campus, making it one of the oldest towns in the United States with a university presence."

In [46]:
result = chat([SystemMessage(content='You are a travel guide who explain places in exactly 5 words.'),
               HumanMessage(content='Can you tell me a fact about Piscataway, NJ?')])

In [47]:
result

AIMessage(content='Historic sites, parks, diverse community.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 39, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e8be3df7-7bef-4ff7-ba1b-d63000aed45d-0')

In [48]:
result.content

'Historic sites, parks, diverse community.'

## InMemoryCache

In [49]:
langchain.llm_cache = InMemoryCache()

In [50]:
llm.predict('tell me one sentence about Somerset,NJ')

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\3958892167.py:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict('tell me one sentence about Somerset,NJ')


'\n\nSomerset, NJ is a diverse and vibrant community located in central New Jersey.'

In [51]:
llm.predict('tell me one sentence about Somerset,NJ')

'\n\nSomerset, NJ is a diverse and vibrant community located in central New Jersey.'

## PromptTemplate

In [52]:
result = llm.generate(["Can you explain in 5 sentences about New Jersey?"]*1)

In [53]:
type(result)

langchain_core.outputs.llm_result.LLMResult

In [54]:
for res in result.generations:
    print(res)

[Generation(text='\n\n1. New Jersey is a state located in the northeastern region of the United States, bordered by New York, Pennsylvania, and Delaware.\n2. It is known for its diverse landscape, with both urban areas like Newark and Jersey City, and scenic beaches and forests in the southern part of the state.\n3. The state has a rich history, being one of the original 13 colonies and playing a significant role in the American Revolution.\n4. New Jersey is also home to many famous landmarks, such as the Atlantic City boardwalk and the Statue of Liberty, which can be seen from its shores.\n5. It is a hub for industry and commerce, with a strong economy driven by industries such as pharmaceuticals, agriculture, and tourism.', generation_info={'finish_reason': 'stop', 'logprobs': None})]


In [55]:
prompt = PromptTemplate.from_template("Can you explain in 5 sentences about {company}?")

In [56]:
chain = LLMChain(llm=llm,prompt=prompt)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\2891286404.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm,prompt=prompt)


In [57]:
chain.run('Google')

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\4104821287.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run('Google')


"\n\n1. Google is a multinational technology company that specializes in internet-related services and products.\n2. It was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University.\n3. Google's most popular service is its search engine, which allows users to search for information on the World Wide Web.\n4. The company has expanded its offerings to include email, online mapping, video sharing, cloud storage, and more.\n5. Google is known for its innovative and user-friendly products, and it has become one of the largest and most influential companies in the world."

In [58]:
chain.run('Open AI')

'\n\n1. Open AI is a non-profit artificial intelligence research organization founded in 2015 by tech entrepreneurs Elon Musk and Sam Altman.\n2. Its mission is to promote and develop friendly AI that benefits humanity as a whole.\n3. Open AI conducts research in various fields of AI, including machine learning, robotics, and natural language processing.\n4. It has developed several notable AI systems, such as GPT-3, which can generate human-like text and DALL-E, which can create images from text descriptions.\n5. Open AI also advocates for responsible and ethical use of AI and has released several tools and resources to promote transparency and safety in AI development.'

In [59]:
no_input_prompt = PromptTemplate(input_variables=[],
                                   template = 'Tell me a fact')

In [60]:
no_input_prompt.format()

'Tell me a fact'

In [61]:
llm(no_input_prompt.format())

'\n\nThe shortest war in history was between Great Britain and Zanzibar in 1896, lasting only 38 minutes.'

In [62]:
single_input_prompt = PromptTemplate(input_variables=['topic'],
                                   template = 'Tell me a fact about {topic}')

In [63]:
llm(single_input_prompt.format(topic='Somerset NJ'))

"\n\nSomerset, NJ is home to the world's largest Hindu temple, the BAPS Shri Swaminarayan Mandir, which was built in 2014 and spans over 162 acres."

In [64]:
multiple_input_prompt = PromptTemplate(
    input_variables=["topic", "level"], 
    template="Tell me a fact about {topic} for a student {level} level."
)
llm(multiple_input_prompt.format(topic='Mars',level='8th Grade'))

'\n\nMars is the fourth planet from the sun and is often referred to as the "Red Planet" due to its reddish appearance caused by iron oxide (rust) on its surface.'

In [65]:
multiple_input_prompt.input_variables

['level', 'topic']

## Serizalization

In [66]:
multiple_input_prompt.save("prompt.json")

In [67]:
from langchain.prompts import load_prompt

In [68]:
loaded_prompt = load_prompt('prompt.json')

In [69]:
loaded_prompt

PromptTemplate(input_variables=['level', 'topic'], input_types={}, partial_variables={}, template='Tell me a fact about {topic} for a student {level} level.')

## Chat Model Templates

In [70]:
system_template="You are an AI recipe assistant that specializes in {dietary_preference} dishes that can be prepared in {cooking_time}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [71]:
system_message_prompt.input_variables

['cooking_time', 'dietary_preference']

In [72]:
human_template="{recipe_request}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [73]:
human_message_prompt.input_variables

['recipe_request']

In [74]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [75]:
chat_prompt.input_variables

['cooking_time', 'dietary_preference', 'recipe_request']

In [76]:
chat_prompt.format_prompt(cooking_time="15 min", dietary_preference="Vegan", recipe_request="Quick Snack").to_messages()

[SystemMessage(content='You are an AI recipe assistant that specializes in Vegan dishes that can be prepared in 15 min.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Quick Snack', additional_kwargs={}, response_metadata={})]

In [77]:
request = chat_prompt.format_prompt(cooking_time="15 min", dietary_preference="Vegan", recipe_request="Quick Snack").to_messages()

In [78]:
chat = ChatOpenAI()

In [79]:
result = chat(request)

In [80]:
result

AIMessage(content="How about making a delicious and quick avocado toast as a snack? Here's a simple recipe for you:\n\nIngredients:\n- 1 ripe avocado\n- 2 slices of bread (use your favorite type)\n- Lemon juice\n- Salt and pepper\n- Red pepper flakes (optional)\n\nInstructions:\n1. Toast the bread slices to your desired level of crunchiness.\n2. While the bread is toasting, mash the ripe avocado in a bowl with a fork.\n3. Add a squeeze of lemon juice, salt, and pepper to the mashed avocado and mix well.\n4. Once the bread is toasted, spread the avocado mixture evenly on top of each slice.\n5. Sprinkle some red pepper flakes on top for an extra kick, if desired.\n6. Enjoy your quick and tasty avocado toast snack!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 163, 'prompt_tokens': 34, 'total_tokens': 197, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_toke

In [81]:
print(result.content)

How about making a delicious and quick avocado toast as a snack? Here's a simple recipe for you:

Ingredients:
- 1 ripe avocado
- 2 slices of bread (use your favorite type)
- Lemon juice
- Salt and pepper
- Red pepper flakes (optional)

Instructions:
1. Toast the bread slices to your desired level of crunchiness.
2. While the bread is toasting, mash the ripe avocado in a bowl with a fork.
3. Add a squeeze of lemon juice, salt, and pepper to the mashed avocado and mix well.
4. Once the bread is toasted, spread the avocado mixture evenly on top of each slice.
5. Sprinkle some red pepper flakes on top for an extra kick, if desired.
6. Enjoy your quick and tasty avocado toast snack!


# Chatbot

In [82]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [83]:
store = {}

In [84]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [85]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [86]:
config = {"configurable": {"session_id": "abc2"}}

In [87]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm JJ")],
    config=config,
)

In [88]:
response.content

'Hello JJ! How are you doing today?'

In [89]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

In [90]:
response.content

'Your name is JJ. How can I assist you today, JJ?'

In [91]:
store

{'abc2': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm JJ", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello JJ! How are you doing today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8397aff9-bf54-4478-8a62-97a57636f2c4-0'), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is JJ. How can I assist you today, JJ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 34, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens'

## Few shot promot

In [92]:
# Create a few-shot prompt template
prompt_template = """
Translate the following English sentences into French:

English: How are you?
French: Comment ça va?

English: What time is it?
French: Quelle heure est-il?

English: I love programming.
French: J'adore la programmation.

English: {sentence}
French:
"""

In [93]:
prompt = PromptTemplate(template=prompt_template, input_variables=["sentence"])

In [94]:
chain = LLMChain(prompt=prompt, llm=llm)

In [95]:
new_sentence = "Where is the library?"

In [96]:
result = chain.run(sentence=new_sentence)

In [97]:
result

'Où est la bibliothèque?'

In [98]:
system_message_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful travel guide who explains different places"
)

In [99]:
example_input_one = HumanMessagePromptTemplate.from_template("Explain about Edison NJ")

In [100]:
example_output_one = AIMessagePromptTemplate.from_template("1. Edison is in teh Northeast corridor. 2. Edison is a very nice place to live. The end")

In [101]:
human_message_prompt = HumanMessagePromptTemplate.from_template("{place_name}")

In [102]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_input_one, example_output_one, human_message_prompt]
)

In [103]:
chat_prompt

ChatPromptTemplate(input_variables=['place_name'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful travel guide who explains different places'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Explain about Edison NJ'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='1. Edison is in teh Northeast corridor. 2. Edison is a very nice place to live. The end'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['place_name'], input_types={}, partial_variables={}, template='{place_name}'), additional_kwargs={})])

In [104]:
place_name= "Explain about Piscataway, NJ"

In [105]:
request = chat_prompt.format_prompt(place_name=place_name).to_messages()

In [106]:
result = chat(request)

In [107]:
result

AIMessage(content="Piscataway is a township in Middlesex County, New Jersey, located in the central part of the state. Here are some key points about Piscataway:\n\n1. Piscataway is known for being home to Rutgers University's main campus, one of the oldest and most prestigious universities in the United States.\n2. The township offers a mix of residential neighborhoods, parks, and commercial areas, making it a diverse and vibrant community.\n3. Piscataway has a rich history, with several historic sites and landmarks, including the East Jersey Olde Towne Village, which showcases early American architecture and lifestyle.\n4. Outdoor enthusiasts can enjoy various recreational opportunities in Piscataway, such as hiking and biking trails, parks, and sports facilities.\n5. The township is conveniently located near major highways, providing easy access to neighboring cities like New Brunswick and Edison, as well as attractions in the greater New York City metropolitan area.\n\nOverall, Pis

In [108]:
result.content

"Piscataway is a township in Middlesex County, New Jersey, located in the central part of the state. Here are some key points about Piscataway:\n\n1. Piscataway is known for being home to Rutgers University's main campus, one of the oldest and most prestigious universities in the United States.\n2. The township offers a mix of residential neighborhoods, parks, and commercial areas, making it a diverse and vibrant community.\n3. Piscataway has a rich history, with several historic sites and landmarks, including the East Jersey Olde Towne Village, which showcases early American architecture and lifestyle.\n4. Outdoor enthusiasts can enjoy various recreational opportunities in Piscataway, such as hiking and biking trails, parks, and sports facilities.\n5. The township is conveniently located near major highways, providing easy access to neighboring cities like New Brunswick and Edison, as well as attractions in the greater New York City metropolitan area.\n\nOverall, Piscataway offers a b

* In the example above, it is not guaranted to have teh response in 2 numbered sentences

## Parsisng output- comma seperated values

In [109]:
output_parser = CommaSeparatedListOutputParser()

In [110]:
format_instructions = output_parser.get_format_instructions()

In [111]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [112]:
reply = "one, two, three"
output_parser.parse(reply)

['one', 'two', 'three']

In [113]:
human_template = '{request} {format_instructions}'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [114]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

chat_prompt.format_prompt(request="Please provide 3 facts about Piscataway, NJ",
                   format_instructions = output_parser.get_format_instructions())

request = chat_prompt.format_prompt(request="Please provide 3 facts about Piscataway, NJ",
                   format_instructions = output_parser.get_format_instructions()).to_messages()

In [115]:
result = chat(request)

In [116]:
type(result.content)

str

In [117]:
result.content

'Home to Rutgers University,,\nLocated in Middlesex County,,\nPiscataway was founded in 1666'

In [118]:
output_formatted = output_parser.parse(result.content)

In [119]:
type(output_formatted)

list

In [120]:
output_formatted[0]

'Home to Rutgers University'

In [121]:
output_formatted

['Home to Rutgers University',
 '',
 '',
 'Located in Middlesex County',
 '',
 '',
 'Piscataway was founded in 1666']

In [122]:
for index, element in enumerate(output_formatted):
    print(f"Element {index}: {element}")

Element 0: Home to Rutgers University
Element 1: 
Element 2: 
Element 3: Located in Middlesex County
Element 4: 
Element 5: 
Element 6: Piscataway was founded in 1666


## Parsisng output- Date time

In [123]:
llm.predict('When was Piscataway,NJ incorporated?')

'\n\nPiscataway, NJ was incorporated as a township on February 21, 1798.'

In [124]:
output_parser = DatetimeOutputParser()

In [125]:
output_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1806-02-23T23:42:14.842308Z, 0501-04-12T15:42:16.992260Z, 0688-03-31T17:29:49.485703Z\n\nReturn ONLY this string, no other words!"

In [126]:
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [127]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [128]:
print(chat_prompt.format(request="When was Piscataway,NJ incorporated?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

Human: When was Piscataway,NJ incorporated?
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1757-06-19T07:28:16.769432Z, 1636-02-04T16:23:49.650764Z, 0139-11-03T08:31:38.462564Z

Return ONLY this string, no other words!


In [129]:
request = chat_prompt.format_prompt(request="When was Piscataway,NJ incorporated?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [130]:
result = chat(request,temperature=0)

In [131]:
result.content

'1666-02-21T00:00:00.000000Z'

## Pydantic JSON Parser- python object conversion of output

In [132]:
from pydantic import BaseModel, Field

In [133]:
class Scientist(BaseModel):
    discoveries: list = Field(description="Python list of discoveries")

In [134]:
query = 'Provide 3 discoveries of Thomas A Edison' 

In [135]:
parser = PydanticOutputParser(pydantic_object=Scientist)

In [136]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["discoveries"]}
```


In [137]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [138]:
input = prompt.format_prompt(query="Provide 3 discoveries of Thomas A Edison")

In [139]:
#output = chat(input)

# Document/Data Connectors

## Loading CSV file

In [140]:
from langchain.document_loaders import CSVLoader

In [141]:
loader = CSVLoader('penguins.csv')

In [142]:
data = loader.load()

In [143]:
type(data)

list

In [144]:
data[0]

Document(metadata={'source': 'penguins.csv', 'row': 0}, page_content='species: Adelie\nisland: Torgersen\nbill_length_mm: 39.1\nbill_depth_mm: 18.7\nflipper_length_mm: 181\nbody_mass_g: 3750\nsex: MALE')

## Loading custom .txt File

In [145]:
file_name = 'constitution.txt'

In [146]:
loader = TextLoader(file_name)

In [147]:
documents = loader.load()

In [148]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [149]:
chunks = text_splitter.split_documents(documents)

In [150]:
embeddings = OpenAIEmbeddings()

In [151]:
vector_store = Chroma.from_documents(chunks, embeddings)

In [152]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

In [153]:
retriever=vector_store.as_retriever()

In [154]:
chain = RetrievalQA.from_chain_type(llm, retriever=retriever )
#crc = ConversationalRetrievalChain.from_llm(llm,retriever)
#st.session_state.crc = crc
#st.success('File uploaded, chunked and embedded successfully')


In [155]:
question = 'what is the age to be a senator?'

In [156]:
response = chain.run(question)

In [157]:
response

'To be a Senator in the United States, a person must have attained the age of thirty years.'

In [158]:
multi_query_retriever = MultiQueryRetriever.from_llm(retriever=retriever,llm=llm)

In [159]:
unique_docs = multi_query_retriever.get_relevant_documents(query=question)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\682342697.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  unique_docs = multi_query_retriever.get_relevant_documents(query=question)


In [160]:
unique_docs

[Document(metadata={'source': 'constitution.txt'}, page_content='person not a natural-born................\n    No person shall be a Senator who shall          1          3       3\n     not have attained the age of thirty\n     years, and been nine years a.........\n    No person shall be a Representative             1          2       2\n     who shall not have attained the age\n     of twenty-five years, and been seven\n     years a..............................\n    Right of citizens to vote shall not be         19    .......  ......\n     denied or abridged by the United\n     States or any State on account of\n     sex. [Amendments]....................\n    Right to vote shall not be denied or           24          1  ......\n     abridged by the United States or any\n     State for failure to pay any poll tax\n     or other tax. [Amendments]...........\n    Right to vote shall not be denied or           26          1  ......\n     abridged by the United States or any\n     State

## ContextualCompressionRetriever

In [161]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [162]:
compressor = LLMChainExtractor.from_llm(llm)

In [163]:
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

In [164]:
docs = vector_store.similarity_search('what is the age to be a senator?')

In [165]:
docs[0]

Document(metadata={'source': 'constitution.txt'}, page_content='person not a natural-born................\n    No person shall be a Senator who shall          1          3       3\n     not have attained the age of thirty\n     years, and been nine years a.........\n    No person shall be a Representative             1          2       2\n     who shall not have attained the age\n     of twenty-five years, and been seven\n     years a..............................\n    Right of citizens to vote shall not be         19    .......  ......\n     denied or abridged by the United\n     States or any State on account of\n     sex. [Amendments]....................\n    Right to vote shall not be denied or           24          1  ......\n     abridged by the United States or any\n     State for failure to pay any poll tax\n     or other tax. [Amendments]...........\n    Right to vote shall not be denied or           26          1  ......\n     abridged by the United States or any\n     State 

In [166]:
compressed_docs = compression_retriever.get_relevant_documents("what is the age to be a senator?")

In [167]:
compressed_docs

[Document(metadata={'source': 'constitution.txt'}, page_content='No person shall be a Senator who shall not have attained the age of thirty years'),
 Document(metadata={'source': 'constitution.txt'}, page_content='No person shall be a Senator who shall not be thirty years of age,'),
 Document(metadata={'source': 'constitution.txt'}, page_content='No person shall be a Senator who shall not have attained the age of thirty years'),
 Document(metadata={'source': 'constitution.txt'}, page_content='No Person shall be a Senator who shall not have attained to the Age of thirty Years')]

## Loading custom PDF File

In [168]:
pdf_name= 'doc_inputs/falcon-users-guide-2021-09.pdf'

In [169]:
pdf_loader = PyPDFLoader(pdf_name)

In [170]:
documents = pdf_loader.load()

Overwriting cache for 0 11250


In [171]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [172]:
chunks = text_splitter.split_documents(documents)

In [173]:
embeddings = OpenAIEmbeddings()

In [174]:
vector_store = Chroma.from_documents(chunks, embeddings)

In [175]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

In [176]:
retriever=vector_store.as_retriever()

In [177]:
chain = RetrievalQA.from_chain_type(llm, retriever=retriever )

In [178]:
question = 'where can I find the standard price for Falcon 9 and Falcon Heavy launch services?'

In [179]:
response = chain.run(question)

In [180]:
response

'The standard price for Falcon 9 and Falcon Heavy launch services can be found at https://www.spacex.com/media/Capabilities&Services.pdf.'

# Loaders

## WebBaseLoader - loads web pages

In [181]:
from langchain_community.document_loaders import WebBaseLoader
from bs4 import BeautifulSoup

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [182]:
loader = WebBaseLoader("https://www.google.com/")

In [183]:
data = loader.load()

In [184]:
type(data[0])

langchain_core.documents.base.Document

In [185]:
soup = BeautifulSoup(data[0].page_content, 'html.parser')

In [186]:
print(soup.prettify())

GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in Advanced searchAdvertisingBusiness SolutionsAbout Google© 2024 - Privacy - Terms



## WikipediaLoader

In [187]:
from langchain.document_loaders import WikipediaLoader

In [188]:
loader = WikipediaLoader(query='New Jersey')
documents = loader.load()

C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [189]:
documents

[Document(metadata={'title': 'New Jersey', 'summary': "New Jersey is the most densely populated U.S. state. The state is located in both the Mid-Atlantic and Northeastern regions of the United States, at the geographic hub of the heavily urbanized Northeast megalopolis. New Jersey is bordered to the northwest, north, and northeast by the State of New York; on its east, southeast, and south by the Atlantic Ocean; on its west by the Delaware River and Pennsylvania; and on its southwest by the Delaware Bay and Delaware. At 7,354 square miles (19,050 km2), New Jersey is the fifth-smallest state in land area, but with over 9.5 million residents as of 2024 U.S. Census Bureau estimates, its highest population ever, the state ranks 11th in population. The state capital is Trenton, and the state's most populous city is Newark. New Jersey is the only U.S. state in which every county is deemed urban by the U.S. Census Bureau.\nNew Jersey was first inhabited by Paleo-Indians as early as 13,000 B.C

In [190]:
len(documents)

23

# Chroma DB Basics

In [191]:
import chromadb

In [192]:
chroma_client = chromadb.Client()

In [193]:
from chromadb.utils import embedding_functions

In [194]:
default_ef = embedding_functions.DefaultEmbeddingFunction()

In [195]:
type(default_ef)

chromadb.utils.embedding_functions.ONNXMiniLM_L6_V2

In [196]:
colection_name = "test_collection"

In [197]:
collection = chroma_client.get_or_create_collection(
    colection_name, embedding_function=default_ef
)

In [198]:
# Define text documents
documents = [
    {"id": "doc1", "text": "Hello, world!"},
    {"id": "doc2", "text": "How are you today?"},
    {"id": "doc3", "text": "Goodbye, see you later!"},
]

In [199]:
for doc in documents:
    collection.upsert(ids=doc["id"], documents=[doc["text"]])

In [200]:
# define a query text
query_text = "Hello, world"

In [201]:
def search_chroma_db(query: str):
    results = collection.query(
        query_texts=[query],  # Corrected the variable name
        n_results=2,
    )
    
    # Iterate over the results with proper indentation
    for idx, document in enumerate(results["documents"][0]):
        doc_id = results["ids"][0][idx]
        distance = results["distances"][0][idx]
        print(
            f"For the query: {query}, \n"
            f"Found similar document: {document} (ID: {doc_id}, Distance: {distance})"
        )

In [202]:
search_chroma_db(query_text)

For the query: Hello, world, 
Found similar document: Hello, world! (ID: doc1, Distance: 0.12731888890266418)
For the query: Hello, world, 
Found similar document: How are you today? (ID: doc2, Distance: 1.263188362121582)


In [203]:
search_chroma_db("age of earth")

For the query: age of earth, 
Found similar document: Hello, world! (ID: doc1, Distance: 1.6677895784378052)
For the query: age of earth, 
Found similar document: Goodbye, see you later! (ID: doc3, Distance: 1.7939949035644531)


In [204]:
search_chroma_db("Hello, world")

For the query: Hello, world, 
Found similar document: Hello, world! (ID: doc1, Distance: 0.12731888890266418)
For the query: Hello, world, 
Found similar document: How are you today? (ID: doc2, Distance: 1.263188362121582)


In [205]:
documents = [    
    {
        "id": "doc4",
        "text": "Microsoft is a technology company that develops software. It was founded by Bill Gates and Paul Allen in 1975.",
    },
]

In [206]:
for doc in documents:
    collection.upsert(ids=doc["id"], documents=[doc["text"]])

In [207]:
query_text = "find document related to technology company"

In [208]:
search_chroma_db(query_text)

For the query: find document related to technology company, 
Found similar document: Microsoft is a technology company that develops software. It was founded by Bill Gates and Paul Allen in 1975. (ID: doc4, Distance: 1.3692371845245361)
For the query: find document related to technology company, 
Found similar document: Goodbye, see you later! (ID: doc3, Distance: 1.9923689365386963)


In [209]:
from chromadb.utils import embedding_functions 

In [210]:
default_ef = embedding_functions.DefaultEmbeddingFunction()

In [211]:
text = "Hello"

In [212]:
emb = default_ef(text)

In [213]:
emb

[[-0.057101599872112274,
  0.09410139173269272,
  -0.042961250990629196,
  0.015190397389233112,
  0.003590941894799471,
  0.019870730116963387,
  0.08121775090694427,
  0.037185512483119965,
  -0.01011547539383173,
  -0.05368809401988983,
  0.061433643102645874,
  -0.09885642677545547,
  -0.02234482765197754,
  -0.06459888070821762,
  -0.01261056587100029,
  -0.0023362324573099613,
  -0.07551310211420059,
  -0.009480354376137257,
  -0.059548620134592056,
  -0.07525584846735,
  -0.06103133037686348,
  0.023791328072547913,
  0.022302620112895966,
  0.017092231661081314,
  -0.0524519644677639,
  0.05123633146286011,
  0.008438421413302422,
  0.062142882496118546,
  -0.019462935626506805,
  -0.09673572331666946,
  -0.002981971949338913,
  0.0025748549960553646,
  0.1014222726225853,
  -0.01917104236781597,
  0.000771118386182934,
  -0.025375474244356155,
  -0.023148272186517715,
  -0.10248353332281113,
  -0.014852477237582207,
  0.018474869430065155,
  -0.03515508025884628,
  -0.03910303

In [214]:
# OpenAIEmbeddings
emb = embeddings.embed_query(text)

In [215]:
emb

[-0.02183269993388962,
 -0.007037575206172415,
 -0.02864325547344561,
 -0.024310444639069517,
 -0.02355804027321282,
 0.0290583757658649,
 -0.01249250732429478,
 -0.0027242227280498345,
 -0.008198613668087017,
 -0.0051079175859896075,
 0.02931782548296565,
 -0.003249608591443376,
 -0.01569671254120844,
 -0.0025685528512232538,
 0.012155222319534758,
 -0.0009583426719376792,
 0.03863207423843755,
 0.005808431915293894,
 0.01896577972173598,
 -0.013698948229416481,
 -0.019796018443929333,
 0.01008611031342884,
 0.005124133309723731,
 0.008860210353561666,
 -0.008127264111127826,
 -0.0053219639285606414,
 0.0025847685749573776,
 -0.012155222319534758,
 0.0033760903518130565,
 -0.01569671254120844,
 0.0038398568841121047,
 -0.016202638651364547,
 -0.01781122838750538,
 -0.012901140954691376,
 0.004102549699393549,
 -0.016267501546301043,
 -0.0009129389481619826,
 -0.00976828436341429,
 0.02149541399780698,
 -0.008458063385187767,
 0.01303086581324175,
 -0.006473271931779892,
 0.00350257211

In [216]:
collection

Collection(name=test_collection)

# ChromaDB file peristence

In [217]:
loader = TextLoader("constitution.txt")
documents = loader.load()

In [218]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

Created a chunk of size 970, which is longer than the specified 500
Created a chunk of size 5997, which is longer than the specified 500
Created a chunk of size 1702, which is longer than the specified 500
Created a chunk of size 684, which is longer than the specified 500
Created a chunk of size 578, which is longer than the specified 500
Created a chunk of size 667, which is longer than the specified 500
Created a chunk of size 645, which is longer than the specified 500
Created a chunk of size 653, which is longer than the specified 500
Created a chunk of size 709, which is longer than the specified 500
Created a chunk of size 635, which is longer than the specified 500
Created a chunk of size 596, which is longer than the specified 500
Created a chunk of size 528, which is longer than the specified 500
Created a chunk of size 650, which is longer than the specified 500
Created a chunk of size 510, which is longer than the specified 500
Created a chunk of size 638, which is longer t

In [219]:
embedding_function = OpenAIEmbeddings()

In [220]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function,persist_directory='chroma_db')

In [221]:
# Helpful to force a save
db.persist()

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\3220725503.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [222]:
db_connection = Chroma(persist_directory='chroma_db/',embedding_function=embedding_function)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\2668171911.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db_connection = Chroma(persist_directory='chroma_db/',embedding_function=embedding_function)


In [223]:
new_doc = "How many amendments are there?"

In [224]:
docs = db_connection.similarity_search(new_doc)

In [225]:
print(docs[0].page_content)

Bail. Excessive bail shall not be                   8    .......  ......
 required, nor excessive fines nor cruel
 and unusual punishments imposed.
 [Amendments].............................
Ballot for President and Vice President.           12    .......  ......
 The electors shall vote by. [Amendments].
Ballot. If no person have a majority of            12    .......  ......
 the electoral votes for President and
 Vice President, the House of
 Representatives shall immediately choose
 the President by. [Amendments]...........
Bankruptcies. Congress shall have power to          1          8       4
 pass uniform laws on the subject of......
Basis of representation among the several          14          2  ......
 States. Provisions relating to the.
 [Amendments].............................
Bear arms shall not be infringed. A well-           2    .......  ......
 regulated militia being necessary to the
 security of a free State, the right of
 the people to keep and. [Amendments].....

In [226]:
# add a new document
loader = TextLoader("custom_text_data.txt")
documents = loader.load()

In [227]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

In [228]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function,persist_directory='chroma_db')

In [229]:
docs = db.similarity_search('asdfg')

In [230]:
docs[0].page_content

'asdfg: testing data'

In [231]:
docs = db.similarity_search('zzzz')

In [232]:
docs[0].page_content

'Q'

In [233]:
docs

[Document(metadata={'source': 'constitution.txt'}, page_content='Q'),
 Document(metadata={'source': 'constitution.txt'}, page_content='J'),
 Document(metadata={'source': 'constitution.txt'}, page_content='J'),
 Document(metadata={'source': 'constitution.txt'}, page_content='J')]

# Create Image

In [234]:
llm = OpenAI(temperature=0.9)
prompt = "Brooklyn bridge in the night"

In [235]:
image_url = DallEAPIWrapper().run(prompt)

AuthenticationError: Error code: 401 - {'error': {'code': 'invalid_api_key', 'message': 'Incorrect API key provided: **********. You can find your API key at https://platform.openai.com/account/api-keys.', 'param': None, 'type': 'invalid_request_error'}}

In [ ]:
image_url

In [ ]:
!(image_url)


In [ ]:
display(Image(url=image_url))

# LLMChains

In [236]:
human_message_prompt = HumanMessagePromptTemplate.from_template(
        "Tell me 3 things about {place}"
    )

In [237]:
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

In [238]:
chat = ChatOpenAI()

In [239]:
chain = LLMChain(llm=chat, prompt=chat_prompt_template)

In [240]:
chain.run(place="Piscataway, NJ")

"1. Piscataway, NJ is located in Middlesex County, New Jersey and is a suburb of New York City. It is known for its diverse population and strong sense of community.\n\n2. Piscataway is home to Rutgers University's main campus, which is the largest university in New Jersey. The university brings a vibrant energy to the town and offers a wide range of cultural and educational opportunities for residents.\n\n3. Piscataway has a rich history, with some parts of the town dating back to the 17th century. The town is home to several historic sites, including the East Jersey Olde Towne Village and the Cornelius Low House Museum, which offer a glimpse into Piscataway's past."

## Simple Sequential Chain

In [241]:
template = "Give me a simple bullet point outline for a blog post on {topic}"
first_prompt = ChatPromptTemplate.from_template(template)
chain_one = LLMChain(llm=llm,prompt=first_prompt)

In [242]:
template = "Write a blog post using this outline: {outline}"
second_prompt = ChatPromptTemplate.from_template(template)
chain_two = LLMChain(llm=llm,prompt=second_prompt)

In [243]:
full_chain = SimpleSequentialChain(chains=[chain_one,chain_two],
                                  verbose=True)

In [244]:
result = full_chain.run("Data Science")
print(result)



> Entering new SimpleSequentialChain chain...



I. Introduction
    A. Explanation of data science
    B. Importance of data science in today's world

II. What is Data Science?
    A. Definition and scope of data science
    B. Key components of data science (data collection, cleaning, analysis, interpretation)

III. Tools and Techniques Used in Data Science
    A. Data mining
    B. Machine learning
    C. Data visualization
    D. Statistical analysis
    E. Programming languages (Python, R, etc.)

IV. Real-World Applications of Data Science
    A. Business and marketing
    B. Healthcare
    C. Education
    D. Transportation
    E. Social media
    F. Other industries

V. Benefits of Data Science
    A. Better decision making
    B. Predictive analytics
    C. Improved efficiency and cost savings
    D. Personalization
    E. Innovation and new opportunities

VI. Challenges in Data Science
    A. Data privacy and security
    B. Data quality and reliability
    C. Talent shortag

## Sequential Chain

In [245]:
template1 = "Give a summary of this employee's performance review:\n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain_1 = LLMChain(llm=llm,
                     prompt=prompt1,
                     output_key="review_summary")

In [246]:
template2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain_2 = LLMChain(llm=llm,
                     prompt=prompt2,
                     output_key="weaknesses")

In [247]:
template3 = "Create a personalized plan to help address and fix these weaknesses:\n{weaknesses}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain_3 = LLMChain(llm=llm,
                     prompt=prompt3,
                     output_key="final_plan")

In [248]:
seq_chain = SequentialChain(chains=[chain_1,chain_2,chain_3],
                            input_variables=['review'],
                            output_variables=['review_summary','weaknesses','final_plan'],
                            verbose=True)

In [249]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [250]:
results = seq_chain(employee_review)



> Entering new SequentialChain chain...


C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\2927280498.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = seq_chain(employee_review)



> Finished chain.


In [251]:
results

{'review': "\nEmployee Information:\nName: Joe Schmo\nPosition: Software Engineer\nDate of Review: July 14, 2023\n\nStrengths:\nJoe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.\n\nOne of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.\n\nJoe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.\n\nAnother notable strength is Joe's adaptability.

In [252]:
print(results['final_plan'])

4. Public speaking - Joe may lack confidence and struggle with public speaking in presentations or meetings.
5. Collaboration - Joe may have difficulty working in a team and may prefer to work independently.
6. Adaptability - Joe may struggle with adapting to change and may have a hard time adjusting to new processes or procedures.

Step 1: Set Clear Expectations and Goals 
Start by setting clear expectations and goals for Joe in each of the areas listed. Make sure they are specific, measurable, achievable, relevant, and time-bound (SMART goals). For example, a goal for Joe's time management weakness could be to complete all tasks on his to-do list by the end of each day for the next month.

Step 2: Provide Training and Resources 
Identify and provide training opportunities for Joe to improve his skills. This could include time management workshops, writing courses, public speaking classes, team building activities, and change management training. Provide him with resources such as boo

## LLMRouterChain

In [253]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router import MultiPromptChain

In [254]:
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms. 
You assume no prior knowledge. Here is the question\n{input}'''

In [255]:
expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a 
PhD level understanding of Physics. Here is the question\n{input}'''

In [256]:
prompt_infos = [
      {'name':'advanced physics','description': 'Answers advanced physics questions',
     'prompt_template':expert_template},
    {'name':'beginner physics','description': 'Answers basic beginner physics questions',
     'prompt_template':beginner_template},
    
]

In [257]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [258]:
#destination_chains

In [259]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [260]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [261]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [262]:
print(destinations_str)

advanced physics: Answers advanced physics questions
beginner physics: Answers basic beginner physics questions


In [263]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [264]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
advanced physics: Answers advanced physics questions
beginner physics: Answers basic beginner physics

In [265]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [266]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

In [267]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [268]:
chain.run("How do magnets work?")



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'How do magnets work?'}
> Finished chain.


"\n\nMagnets work through a force called magnetic force. This force is created by the movement of tiny particles called electrons within the magnet. These electrons have a property called spin, which is like a tiny spinning top. When many of these spinning electrons are moving in the same direction, they create a magnetic field around the magnet.\n\nThe magnetic field is an invisible force that extends out from the magnet in all directions. When another magnet or certain metals, such as iron, come into contact with this magnetic field, they can feel the force and either attract or repel each other. This is why magnets can stick to metal objects or each other.\n\nBut why do some magnets attract and others repel? This depends on the orientation of the magnetic field. Just like how a compass needle always points north because of the Earth's magnetic field, magnets also have a north and south pole. These poles can either attract or repel each other depending on their orientation. Opposite 

In [269]:
chain.run("How do Feynman Diagrams work?")



> Entering new MultiPromptChain chain...


OutputParserException: Parsing text
```json

 raised following error:
Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

## TransformChain

In [271]:
from langchain.chains import TransformChain

In [272]:
yelp_review = open('yelp_review.txt').read()

In [273]:
print(yelp_review)

TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!

REVIEW:
OH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!

FIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.

NOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!

THEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!

THE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND EXTREMELY KN

In [274]:
yelp_review.split('REVIEW:')[-1]

"\nOH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!\n\nFIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.\n\nNOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!\n\nTHEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!\n\nTHE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND EXTREMELY KNOWLEDGEABLE ABOUT THE MENU. THEY WENT ABOVE 

In [275]:
def transformer_fun(inputs: dict) -> dict:
    '''
    Notice how this always takes an inputs dictionary.
    Also outputs a dictionary. You can call the output and input keys whatever you want, 
    just make sure to reference it correct in the chain call.
    '''   
    text = inputs['text']
    only_review_text = text.split('REVIEW:')[-1]
    lower_case_text = only_review_text.lower()
    return {'output':lower_case_text}

In [276]:
transform_chain = TransformChain(input_variables=['text'],
                                 output_variables=['output'],
                                 transform=transformer_fun)

In [277]:
template = "Create a one sentence summary of this review:\n{review_text}"

In [278]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(template)
summary_chain = LLMChain(llm=llm,
                     prompt=prompt,
                     output_key="review_summary")

In [279]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain,summary_chain],
                                        verbose=True)


In [280]:
result = sequential_chain(yelp_review)



> Entering new SimpleSequentialChain chain...

oh my goodness, where do i begin? this restaurant is absolutely phenomenal! i went there last night with my friends, and we were blown away by the experience!

first of all, the ambiance is out of this world! the moment you step inside, you're greeted with a warm and inviting atmosphere. the decor is stunning, and it immediately sets the tone for an unforgettable dining experience.

now, let's talk about the food! wow, just wow! the menu is a paradise for food lovers. every dish we ordered was a masterpiece. the flavors were bold, vibrant, and exploded in our mouths. from starters to desserts, every bite was pure bliss!

their seafood platter is a must-try! the freshness of the seafood is unmatched, and the presentation is simply stunning. i have never tasted such delicious and perfectly cooked seafood in my life. it's a seafood lover's dream come true!

the service was exemplary. the staff was attentive, friendly, and extremely knowledg

In [281]:
result['output']

'This review raves about the phenomenal ambiance, exquisite food, top-notch service, and delectable desserts at a hidden gem of a restaurant.'

In [282]:
result['input']

"TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!\n\nREVIEW:\nOH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!\n\nFIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.\n\nNOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!\n\nTHEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!\n\nTHE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND 

## LLMMathChain

In [283]:
from langchain import LLMMathChain

In [284]:
llm_math_model = LLMMathChain.from_llm(llm)

In [285]:
llm_math_model("What is 17 raised to the power of 11?")

{'question': 'What is 17 raised to the power of 11?',
 'answer': 'Answer: 34271896307633'}

#  OpenAI Functions API

In [286]:
from langchain.chains.openai_functions import create_openai_fn_chain,create_structured_output_chain

In [287]:
class Scientist():
    
    def __init__(self,first_name,last_name):
        self.first_name = first_name
        self.last_name = last_name

In [288]:
json_schema = {"title": "Scientist",
               "description": "Information about a famous scientist",
               "type": "object",
               "properties":{
                   "first_name":{'title':'First Name',
                                 'description': "First name of scientist",
                                 "type": "string"},
                   "last_name":{'title':'Last Name',
                                 'description': "Last name of scientist",
                                 "type": "string"},
               },
                "required": ['first_name','last_name']
              }

In [289]:
template = 'Name a famous {country} scientist'
# human_prompt = HumanMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_template(template)

In [290]:
chain = create_structured_output_chain(json_schema,llm,chat_prompt,verbose=True)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\2350241055.py:1: LangChainDeprecationWarning: The function `create_structured_output_chain` was deprecated in LangChain 0.1.1 and will be removed in 1.0. Use :meth:`~ChatOpenAI.with_structured_output` instead.
  chain = create_structured_output_chain(json_schema,llm,chat_prompt,verbose=True)


In [291]:
result = chain.run(country='Indian')



> Entering new LLMChain chain...
Prompt after formatting:
Human: Name a famous Indian scientist

> Finished chain.


In [292]:
result

{'first_name': 'C.', 'last_name': 'V. Raman'}

# Memory / History

## ChatMessageHistory

In [293]:
history = ChatMessageHistory()

In [294]:
history.add_user_message("Hello, nice to meet you.")

In [295]:
history.add_ai_message("Nice to meet you too!")

In [296]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, nice to meet you.', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you too!', additional_kwargs={}, response_metadata={})])

In [297]:
history.messages

[HumanMessage(content='Hello, nice to meet you.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice to meet you too!', additional_kwargs={}, response_metadata={})]

## ConversationBufferMemory

In [298]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\1750532219.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [299]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\691703142.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [300]:
conversation.predict(input="Hello, nice to meet you!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, nice to meet you!
AI:

> Finished chain.


"Hello! It's nice to meet you too. I am an AI designed to assist with any questions or tasks you may have. How can I help you today?"

In [301]:
conversation.predict(input="Tell me about the Einstein-Szilard Letter ")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, nice to meet you!
AI: Hello! It's nice to meet you too. I am an AI designed to assist with any questions or tasks you may have. How can I help you today?
Human: Tell me about the Einstein-Szilard Letter 
AI:

> Finished chain.


'The Einstein-Szilard Letter was a letter written by physicist Albert Einstein to President Franklin D. Roosevelt in 1939. In the letter, Einstein and physicist Leo Szilard warned the President about the potential development of nuclear weapons by Nazi Germany and urged the United States to start its own nuclear research program. This letter ultimately led to the establishment of the Manhattan Project, which resulted in the development of the atomic bomb during World War II.'

In [302]:
memory.buffer

"Human: Hello, nice to meet you!\nAI: Hello! It's nice to meet you too. I am an AI designed to assist with any questions or tasks you may have. How can I help you today?\nHuman: Tell me about the Einstein-Szilard Letter \nAI: The Einstein-Szilard Letter was a letter written by physicist Albert Einstein to President Franklin D. Roosevelt in 1939. In the letter, Einstein and physicist Leo Szilard warned the President about the potential development of nuclear weapons by Nazi Germany and urged the United States to start its own nuclear research program. This letter ultimately led to the establishment of the Manhattan Project, which resulted in the development of the atomic bomb during World War II."

In [303]:
memory.load_memory_variables({})

{'history': "Human: Hello, nice to meet you!\nAI: Hello! It's nice to meet you too. I am an AI designed to assist with any questions or tasks you may have. How can I help you today?\nHuman: Tell me about the Einstein-Szilard Letter \nAI: The Einstein-Szilard Letter was a letter written by physicist Albert Einstein to President Franklin D. Roosevelt in 1939. In the letter, Einstein and physicist Leo Szilard warned the President about the potential development of nuclear weapons by Nazi Germany and urged the United States to start its own nuclear research program. This letter ultimately led to the establishment of the Manhattan Project, which resulted in the development of the atomic bomb during World War II."}

In [304]:
memory.save_context({"input": "Very Interesting."}, 
                    {"output": "Yes, it was my pleasure as an AI to answer."})

In [305]:
memory.load_memory_variables({})

{'history': "Human: Hello, nice to meet you!\nAI: Hello! It's nice to meet you too. I am an AI designed to assist with any questions or tasks you may have. How can I help you today?\nHuman: Tell me about the Einstein-Szilard Letter \nAI: The Einstein-Szilard Letter was a letter written by physicist Albert Einstein to President Franklin D. Roosevelt in 1939. In the letter, Einstein and physicist Leo Szilard warned the President about the potential development of nuclear weapons by Nazi Germany and urged the United States to start its own nuclear research program. This letter ultimately led to the establishment of the Manhattan Project, which resulted in the development of the atomic bomb during World War II.\nHuman: Very Interesting.\nAI: Yes, it was my pleasure as an AI to answer."}

In [306]:
# write to pickle
with open('memory.pkl','wb') as f:
    f.write(pickle.dumps(conversation.memory))

In [307]:
# read back from pickle file
llm = ChatOpenAI(temperature=0.0)
reload_conversation = ConversationChain(
    llm=llm, 
    memory = pickle.loads(open('memory.pkl','rb').read()),
    verbose=True
)

In [308]:
reload_conversation.memory.buffer

"Human: Hello, nice to meet you!\nAI: Hello! It's nice to meet you too. I am an AI designed to assist with any questions or tasks you may have. How can I help you today?\nHuman: Tell me about the Einstein-Szilard Letter \nAI: The Einstein-Szilard Letter was a letter written by physicist Albert Einstein to President Franklin D. Roosevelt in 1939. In the letter, Einstein and physicist Leo Szilard warned the President about the potential development of nuclear weapons by Nazi Germany and urged the United States to start its own nuclear research program. This letter ultimately led to the establishment of the Manhattan Project, which resulted in the development of the atomic bomb during World War II.\nHuman: Very Interesting.\nAI: Yes, it was my pleasure as an AI to answer."

# Agents

In [309]:
llm = OpenAI(temperature=0)

In [310]:
tools = load_tools(["llm-math"], llm=llm)

In [311]:
dir(AgentType)

['CHAT_CONVERSATIONAL_REACT_DESCRIPTION',
 'CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'CONVERSATIONAL_REACT_DESCRIPTION',
 'OPENAI_FUNCTIONS',
 'OPENAI_MULTI_FUNCTIONS',
 'REACT_DOCSTORE',
 'SELF_ASK_WITH_SEARCH',
 'STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'ZERO_SHOT_REACT_DESCRIPTION',
 '__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__members__',
 '__mod__',
 '__module__',
 '__mul__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',


In [312]:
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_21540\840442388.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(tools,


In [313]:
agent.run("What is 2 times 3?")



> Entering new AgentExecutor chain...
 I should use a calculator to solve this problem.
Action: Calculator
Action Input: 2 * 3
Observation: Answer: 6
Thought: I now know the final answer.
Final Answer: 6

> Finished chain.


'6'

In [314]:
agent.run("What is 2 times 3? What is that number multiplied by 5?")



> Entering new AgentExecutor chain...
 I should use a calculator to solve this problem.
Action: Calculator
Action Input: 2 * 3
Observation: Answer: 6
Thought: I should use the result from the previous calculation as the input for the next calculation.
Action: Calculator
Action Input: 6 * 5
Observation: Answer: 30
Thought: I now know the final answer.
Final Answer: 30

> Finished chain.


'30'

## Python Agent Tools

In [315]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.tools.python.tool import PythonREPLTool

ModuleNotFoundError: No module named 'langchain_experimental'

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent

In [ ]:
agent = create_python_agent(tool=PythonREPLTool(),
                         llm=llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True)

In [ ]:
python_list = [7,3,1,2]

In [ ]:
agent.run(f'''Sort this Python list in descending order {python_list}''')

## Custom Tool

In [ ]:
from langchain.agents import tool

In [316]:
@tool
def first_verse(text: str) -> str:
    '''Returns the first verse in the book named jmj_qwerty'''
    return "Test content"

NameError: name 'tool' is not defined

In [317]:
tools = load_tools(["llm-math"], llm=llm) 
tools = tools +[first_verse]

NameError: name 'first_verse' is not defined

In [318]:
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True)

In [319]:
agent.run("what is the first verse in the book named jmj_qwerty?")



> Entering new AgentExecutor chain...
 I should use a calculator to find the first verse
Action: Calculator
Action Input: jmj_qwerty

ValueError: LLMMathChain._evaluate("
"jmj_qwerty"
") raised error: data type must provide an itemsize. Please try again with a valid numerical expression

In [ ]:
agent.run("what is the first verse in the Gospel of John?")

## Conversation Agents

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
agent_chain = initialize_agent(tools, 
                               llm,
                               agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
                               verbose=True, 
                               memory=memory)

In [ ]:
agent_chain.run(input="What are some good thai food recipes?")

In [ ]:
agent_chain.run("Which one of those dishes is the spiciest?")

# Metadata query

In [320]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())

In [321]:
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

In [322]:
document_content_description = "Brief summary of a movie"

In [323]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

In [324]:
retriever.invoke("I want to watch a movie rated higher than 8.5")

[Document(metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea')]

In [325]:
retriever.invoke("How many movies has rating higher than 9?")

[Document(metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979}, page_content='Three men walk into the Zone, three men walk out of the Zone')]

# Hugging Face

In [326]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [327]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [328]:
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

In [329]:
# encode context the generation is conditioned on
model_inputs = tokenizer('Some facts about New Jesey', return_tensors='pt')

In [330]:
# generate 40 new tokens
greedy_output = model.generate(**model_inputs, max_new_tokens=40)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [331]:
greedy_output

tensor([[ 4366,  6419,   546,   968,   449,  2771,    88,    25,   198,   198,
          3791,   449,  2771,    88,   318,   257,  1402,  3240,   287,   262,
          3504,   286,   262, 13797,  5866,    13,   632,   318,  5140,   287,
           262,  3504,   286,   262, 13797,  5866,    11,   546,  1542,  4608,
          5366,   286,   262, 13797,  5866,    13,   632]])

In [332]:
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Some facts about New Jesey:

New Jesey is a small town in the middle of the Mississippi River. It is located in the middle of the Mississippi River, about 30 miles south of the Mississippi River. It


## Hugging Fcae- Accessing Pre-trained Models Using Pipeline

In [333]:
from transformers import pipeline

pipe = pipeline("text-generation", model="gpt2")

Device set to use cpu


pipe('Gospel of John begins with', max_length=30, num_return_sequences=1)

In [334]:
pipe('Some facts about New York', max_length=30, num_return_sequences=5)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Some facts about New York City are documented here.\n\nFor $1,999, you get to find what it's like to be an intern"},
 {'generated_text': 'Some facts about New York City from around the world:\n\nIn a series of recent media interviews, the mayor, who was sworn in as the'},
 {'generated_text': 'Some facts about New York City as a city. The city, to quote the National Weather Service, "is an industrial city surrounded by natural light,'},
 {'generated_text': 'Some facts about New York:\n\nA few years ago, people asked about what would happen if the subway left Broadway: Instead of an open window'},
 {'generated_text': "Some facts about New York's most controversial mayoral candidate\n\nThe most well known politician in the city's history could take a backseat to another city"}]

In [335]:
from transformers import GPT2Config, GPT2Model

# Building the config
config = GPT2Config()

In [336]:
config

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.47.0",
  "use_cache": true,
  "vocab_size": 50257
}

## Hugging Face with Langchain

In [5]:
from langchain.llms import HuggingFaceEndpoint
from langchain.llms import HuggingFaceHub

## Hugging Face with PyTorch

In [6]:
import torch

In [7]:
torch.__version__

'2.2.2'

In [8]:
llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2")


C:\Users\jimmy\AppData\Local\Temp\ipykernel_13360\1862325830.py:1: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2")


In [9]:
# The LLM takes a prompt as an input and outputs a completion
our_query = 'Some facts about New Jesey'

#Last week langchain has recommended to use invoke function for the below please :)
completion = llm.invoke(our_query)

In [10]:
print(completion)

:

1. New Jersey is the 11th-most populous and the 5th-smallest state in the United States.
2. The capital city of New Jersey is Trenton, and the largest city is Newark.
3. New Jersey is bordered by New York to the north and east, Pennsylvania to the west, and Delaware and the Atlantic Ocean to the south.
4. New Jersey has a diverse economy, with major industries including healthcare, pharmaceuticals, finance, education, technology, and tourism.
5. New Jersey is known for its beautiful beaches, scenic coastline, and historic sites. Some popular attractions include the Jersey Shore, Liberty State Park, Princeton University, and the Grounds for Sculpture.
6. New Jersey is home to several professional sports teams, including the New York Giants and Jets (football), the New Jersey Devils (hockey), and the New Jersey Nets (basketball).
7. New Jersey is the birthplace of several famous people, including Bruce Springsteen, Jon Bon Jovi, and Whitney Houston.
8. The official state motto of New 

## Tokenizers

In [343]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

In [344]:
raw_inputs =     [
        "I love deep learning!",
        "I hate this so much!",
    ]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[   0,  100,  657, 1844, 2239,  328,    2,    1],
        [   0,  100, 4157,   42,   98,  203,  328,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


In [345]:
print('Tokenizer output for "I love deep learning!"')
print(f"Input ids: {inputs['input_ids'][0]}")
print(f"Attention Mask: {inputs['attention_mask'][0]}")
print("-"*100)
print('Tokenizer output for "I hate this so much!"')
print(f"Input ids: {inputs['input_ids'][1]}")
print(f"Attention Mask: {inputs['attention_mask'][1]}")

Tokenizer output for "I love deep learning!"
Input ids: tensor([   0,  100,  657, 1844, 2239,  328,    2,    1])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 0])
----------------------------------------------------------------------------------------------------
Tokenizer output for "I hate this so much!"
Input ids: tensor([   0,  100, 4157,   42,   98,  203,  328,    2])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1])


In [346]:
tokenizer('In the beginning was the Word, and the Word was with God,and the Word was God.')

{'input_ids': [0, 1121, 5, 1786, 21, 5, 15690, 6, 8, 5, 15690, 21, 19, 1840, 6, 463, 5, 15690, 21, 1840, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [347]:
tokens = tokenizer.tokenize('In the beginning was the Word, and the Word was with God,and the Word was God.')

In [348]:
tokens

['In',
 'Ġthe',
 'Ġbeginning',
 'Ġwas',
 'Ġthe',
 'ĠWord',
 ',',
 'Ġand',
 'Ġthe',
 'ĠWord',
 'Ġwas',
 'Ġwith',
 'ĠGod',
 ',',
 'and',
 'Ġthe',
 'ĠWord',
 'Ġwas',
 'ĠGod',
 '.']

In [349]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
token_ids

[1121,
 5,
 1786,
 21,
 5,
 15690,
 6,
 8,
 5,
 15690,
 21,
 19,
 1840,
 6,
 463,
 5,
 15690,
 21,
 1840,
 4]

In [350]:
decoded_tokens = tokenizer.decode(token_ids)

In [351]:
decoded_tokens

'In the beginning was the Word, and the Word was with God,and the Word was God.'

In [352]:
model_prepped_ids = tokenizer.prepare_for_model(token_ids)
model_prepped_ids

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': [0, 1121, 5, 1786, 21, 5, 15690, 6, 8, 5, 15690, 21, 19, 1840, 6, 463, 5, 15690, 21, 1840, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Pipeline

In [353]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I love deep learning!",
        "I hate this so much!",
        'In the beginning was the Word, and the Word was with God,and the Word was God.',
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998645782470703},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455},
 {'label': 'POSITIVE', 'score': 0.9798815250396729}]

In [354]:
text_generator = pipeline("text-generation")

text_generator([
    'In the beginning was the Word, and the Word was with God',
    'When two objects in space get close to each other'
])

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[[{'generated_text': 'In the beginning was the Word, and the Word was with God. Then all the Word became in God, till the whole face became made light, and the whole face became set in God. But then the whole face, and all the face became'}],
 [{'generated_text': 'When two objects in space get close to each other, the two objects get hit by different collisions.\n\nThe first attack is called orbital collisions, or ARCS. These two objects are called gravitational objects, because their orbit is perpendicular to the ground'}]]

In [355]:
summarizer = pipeline("summarization")

summarizer([
    """A Fibonacci heap is a collection of trees satisfying the min-heap property. It allows faster amortized time for many operations than binary or binomial heaps.
    Trees in a Fibonacci heap can have any shape, which facilitates efficient operations. Lazy strategies are employed: node removals and consolidations are delayed until
    absolutely necessary (like during an extract-min operation). The main advantage lies in decreasing a key and merging two heaps, which are constant and amortized
    constant time, respectively. Nodes have a "mark" indicating if they've lost a child since the last time they were made a child of another node, assisting in
    restructuring during operations."""
])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'summary_text': ' A Fibonacci heap is a collection of trees satisfying the min-heap property . It allows faster amortized time for many operations than binary or binomial heaps . Nodes have a "mark" indicating if they\'ve lost a child since the last time they were made a child of another node .'}]

## Model Embeddings

In [356]:
from transformers import AutoModel

model = AutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [357]:
inputs = tokenizer('Trenton is the capital of New Jersey', padding=True, truncation=True, return_tensors='pt')

outputs = model(**inputs)

print(outputs.last_hidden_state.shape) # the token embeddings

torch.Size([1, 11, 768])


In [358]:
# to get the full context vector for the sequence
context_vectors = outputs.last_hidden_state.mean(dim=1)
context_vectors.shape

torch.Size([1, 768])

## Hugging Face Datasets

In [359]:
from datasets import load_dataset

dataset = load_dataset("fka/awesome-chatgpt-prompts")
dataset

ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
dataset = load_dataset("samsum")
dataset

In [ ]:
# Reload GPT Prompt dataset (Cache keeps from loading it twice)
dataset = load_dataset("fka/awesome-chatgpt-prompts")
dataset

In [ ]:
dataset['train'][0]

In [ ]:
# shuffle & sample

dataset = dataset['train'].shuffle(seed=37).select(range(100))
dataset

In [ ]:
# Create Test Dataset

dataset = dataset.train_test_split(train_size=0.8, seed=42)
dataset

## Creating and uploading custom dataset into Hugging Face

In [ ]:
# the sgm files are what contains the articles
from bs4 import BeautifulSoup

# Open the file and parse its content with BeautifulSoup
reuters_articles = []
for i in range(22):
  if i < 10:
    i = f"0{i}"

  # load file data
  with open(f"reuters21578/reut2-0{i}.sgm", 'r', encoding='latin-1') as file:
      soup = BeautifulSoup(file, "html.parser")

  # Extract articles' titles and bodies
  articles = []
  for reuters in soup.find_all('reuters'):
      title = reuters.title.string if reuters.title else ""
      body = reuters.body.string if reuters.body else ""
      articles.append({
            'title': title,
            'body': body
        })

  reuters_articles.extend(articles)

In [ ]:
# Print out the first few articles for inspection
for i, article in enumerate(reuters_articles[:5]):
  print(article)
  print("-"*100)

In [ ]:
len(reuters_articles)

In [ ]:
import json

TRAIN_PCT, VALID_PCT = 0.8, 0.1

# Split the data
train_articles = reuters_articles[:int(len(reuters_articles)*TRAIN_PCT)]
valid_articles = reuters_articles[int(len(reuters_articles)*TRAIN_PCT): int(len(reuters_articles)*(TRAIN_PCT + VALID_PCT))]
test_articles = reuters_articles[int(len(reuters_articles)*(TRAIN_PCT + VALID_PCT)):]

# Function to save articles as JSONL
def save_as_jsonl(data, filename):
    with open(filename, "w") as f:
        for article in data:
            f.write(json.dumps(article) + "\n")

# Save them into temporary JSONL files
save_as_jsonl(train_articles, "train.jsonl")
save_as_jsonl(valid_articles, "valid.jsonl")
save_as_jsonl(test_articles, "test.jsonl")

In [ ]:
# Load them as datasets
data_files = {"train": "train.jsonl", "validation": "valid.jsonl", "test": "test.jsonl"}
dataset = load_dataset("json", data_files=data_files)

In [ ]:
dataset

In [ ]:
dataset['train'][0]

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
dataset

In [ ]:
dataset.push_to_hub("test_dataset1")

## Downloading custom dataset from Hugging Face

In [ ]:
dataset = load_dataset("jimmypjoy/test_dataset1")

In [ ]:
dataset

In [ ]:
# Add a new column to this dataset
def create_full_article_col(example):

  return {'full_article': f"TITLE:{example['title']}\n\nBODY:{example['body']}"}

dataset = dataset.map(create_full_article_col)
dataset

In [ ]:
dataset['train'][0]['full_article']

In [ ]:
type(dataset)

## Creating custom Tokenizer

In [ ]:
# Create a batched dataset for training, creates an iterator object for later usage when training tokenizer

training_corpus = (
    dataset["train"][i : i + 1000]["full_article"]
    for i in range(0, len(dataset["train"]), 1000)
)

In [ ]:
old_tokenizer = AutoTokenizer.from_pretrained("gpt2") # train gpt2 tokenizer

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000) # vocab size of 52000

In [ ]:
example = dataset['test'][2]['full_article']
example

In [ ]:
old_tokenizer.tokenize(example)

In [ ]:
tokenizer.tokenize(example)

In [ ]:
notebook_login()

In [ ]:
tokenizer.push_to_hub("gpt2-reuters-tokenizer")

In [ ]:
# Use newly created tokenizer
tokenizer = AutoTokenizer.from_pretrained("jimmypjoy/gpt2-reuters-tokenizer")

In [ ]:
example = dataset['test'][2]
example

In [ ]:
tokenizer.tokenize(example['full_article'])

## Training gpt2 from scratch in Hugging Face 

In [ ]:
dataset = load_dataset("jimmypjoy/test_dataset1")
dataset

In [ ]:
def create_full_article_col(example):

  return {'full_article': f"TITLE:{example['title']}\n\nBODY:{example['body']}"}

dataset = dataset.map(create_full_article_col)
dataset

In [ ]:
dataset['train'][0]['full_article']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jimmypjoy/gpt2-reuters-tokenizer")

In [ ]:
CONTEXT_LENGTH = 512

def tokenize(element):
    outputs = tokenizer(
        element["full_article"],
        truncation=True,
        max_length=CONTEXT_LENGTH,
        return_overflowing_tokens=False
    )

    return outputs


tokenized_datasets = dataset.map(
    tokenize, batched=True, remove_columns=dataset["train"].column_names
)
tokenized_datasets

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=CONTEXT_LENGTH,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
config

In [ ]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
notebook_login()

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./reuters-gpt2-text-gen",  # local directory
    hub_model_id="ingeniumacademy/reuters-gpt2-text-gen",  # identifier on the Hub
    evaluation_strategy="epoch",
    auto_find_batch_size=True,
    num_train_epochs=2,
    gradient_accumulation_steps=8,
    weight_decay=0.1,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    fp16=True,
    push_to_hub=True,
    logging_steps=10
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
# Using Our Model In Pipeline
import torch
from transformers import pipeline

#pipe = pipeline(
#    "text-generation", model="jimmypjoy/reuters-gpt2-text-gen"
#)

pipe = pipeline(
    "text-generation", model="ingeniumacademy/reuters-gpt2-text-gen"
)

In [ ]:
sample = dataset['test'][2]
sample

In [ ]:
prompt = f"TITLE:{sample['title']}\n\nBODY:"
pipe(prompt, max_new_tokens=128)

In [ ]:
prompt = f"TITLE:{sample['title']}"
pipe(prompt, max_new_tokens=128)

# Langsmith

In [ ]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
#os.environ['LANGCHAIN_PROJECT'] = "pr-drab-gift-52"

In [ ]:
llm = ChatOpenAI()
llm.invoke("Hello, world!")

In [ ]:
llm.invoke("How is your day?")

# LangChain Expression Language- LCEL 

In [360]:
# Step 1: Define classification and answer templates
classification_prompt = PromptTemplate(
    input_variables=["question"],
    template="Classify this question as 'simple' or 'complex': {question}"
)

In [361]:
simple_answer_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer this simple question: {question}"
)

In [362]:
clarification_prompt = PromptTemplate(
    input_variables=["question"],
    template="Ask a clarifying question for this complex question: {question}"
)

In [363]:
# Step 2: Define chains
classify_chain = LLMChain(llm=llm, prompt=classification_prompt)
simple_answer_chain = LLMChain(llm=llm, prompt=simple_answer_prompt)
clarification_chain = LLMChain(llm=llm, prompt=clarification_prompt)

In [364]:
# Step 3: Workflow to decide based on classification
def question_answering_pipeline(question):
    classification_result = classify_chain.run({"question": question}).strip().lower()
    
    print("Question: "+ question +"     Classified as: "+ classification_result )

    if classification_result == "simple":
        answer = simple_answer_chain.run({"question": question})
        print(f"Answer: {answer}")
    else:
        clarification = clarification_chain.run({"question": question})
        print(f"Clarification: {clarification}")

In [365]:
question_answering_pipeline("What is the capital of France?")

Question: What is the capital of France?     Classified as: simple. this question is simple because it only requires the recall of one fact, the capital city of france, and does not involve any reasoning or analysis.
Clarification:  What are the cultural, historical, and economic reasons that make Paris an important city?

Answer:

The capital city of France is Paris. Paris is not only the capital but also the most populous city in France. It is located in the north-central part of the country along the Seine River. Paris is an important city for various reasons, both culturally, historically, and economically.

Culturally, Paris is known as the "City of Light" and the "City of Love." It is famous for its art, fashion, gastronomy, and culture. Paris is home to many world-renowned museums, galleries, and monuments. The Louvre Museum, Musée d'Orsay, and Centre Pompidou are just a few examples. Paris is also famous for its fashion industry, with haute couture designers, fashion houses, an

In [366]:
question_answering_pipeline("Explain the implications of quantum computing.")

Question: Explain the implications of quantum computing.     Classified as: answer: this question is complex, as the implications of quantum computing are extensive and still being explored. some potential implications include the ability to solve certain problems much faster than classical computers, the potential to simulate quantum systems, and the potential for significant advances in areas such as cryptography, optimization, and machine learning. however, there are also challenges associated with building and programming quantum computers, and it is still an open question how quantum computers will be used in practice.
Clarification: 

To answer the complex question fully, let me first provide a brief explanation of quantum computing. Quantum computing is a type of computing technology that uses the principles of quantum mechanics to perform calculations. In classical computing, data is stored and processed in bits, which can represent either a 0 or a 1. In quantum computing, data

# OpenGen Framework

In [367]:
import opengen
print(dir(opengen))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'builder', 'config', 'constraints', 'definitions', 'functions', 'opengen', 'tcp']


In [368]:
from opengen import ImageGenerator

ImportError: cannot import name 'ImageGenerator' from 'opengen' (C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\opengen\__init__.py)

# Delete